# Analiza ryzyka ataków zwierząt w korelacji z gęstością zaludnienia w Australii
Autorzy: Bartosz Kołaciński 251554, Wiktor Pankanin 251606, Edyta Szymańska 251647


## Cel
cel lala


## Założenia projektu
opisdanieadkf


## Opis danych

#### Ataki krokodyli: croc_attacks.csv
Zbiór zawiera historyczne dane dotyczące incydentów z udziałem krokodyli w Australii. Dane w bazie są światowe od 2015 roku, jednak dla naszego projektu ograniczyliśmy je do terenów Australii. Źródło: https://crocattack.org/database/
- species: Gatunek krokodyla
- species_id: Czy gatunek został potwierdzony
- lat / long: Współrzędne geograficzne (szerokość i długość) miejsca ataku
- is_fatal: Zmienna określająca skutek ataku (1 = śmierć, 0 = przeżycie)
- sex: Płeć ofiary (Male = mężczyzna, Female = kobieta, Unknown = nieznana)
- date: Data ataku w formacie YYYY-MM-DD

#### Ataki rekinów: shark_attacks.xlsx
Zbiór zawiera historyczne dane dotyczące incydentów z udziałem rekinów w Australii. Dane w bazie są światowe od 1791 roku, jednak dla naszego projektu ograniczyliśmy je do terenów Australii i lat od 2015 roku. Źródło: https://zenodo.org/records/16355384
- UIN: Unikalny identyfikator ataku
- Incident.month: Miesiąc ataku
- Incident.year: Rok ataku
- Victim.injury: Rodzaj obrażeń ofiary
- State: Stan Australii, w którym doszło do ataku
- Location: Lokalizacja ataku
- Latitude / Longitude: Współrzędne geograficzne (szerokość i długość) miejsca ataku
- Site.category: Kategoria miejsca ataku
- Shark.common.name: Nazwa gatunku rekina
- Shark.scientific.name: Nazwa naukowa gatunku rekina
- Provoked/unprovoked: Czy atak był sprowokowany

#### Gęstość zaludnienia: population_density.gpkg
Zbiór zawiera dane o gęstości zaludnienia w Australii w formacie geopackage. Źródło: https://www.abs.gov.au/statistics/people/population/regional-population/2023-24#data-downloads
- geometry: Cyfrowe granice obszarów w standardzie GDA2020.
- ERP (Estimated Resident Population): Szacunkowa liczba ludności rezydującej (dane historyczne od 2001 r. do obecnych).
- Population Density: Gęstość zaludnienia dla bieżącego roku wyrażona w osobach na kilometr kwadratowy.
- Area_sqkm: Powierzchnia obszaru w kilometrach kwadratowych.
- Components of Change: Składniki zmiany demograficznej (urodzenia, zgony, migracje wewnętrzne i zagraniczne).

### Potrzebne biblioteki
- folium : biblioteka do tworzenia interaktywnych map w formacie HTML
- geopandas : biblioteka do pracy z danymi geograficznymi
- pandas : biblioteka do analizy i manipulacji danymi
- branca : biblioteka do tworzenia kolorowych map w folium

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from branca.colormap import LinearColormap
from folium.plugins import HeatMap

### Wczytywanie danych

In [ ]:
def read_shark_data(path: str) -> gpd.GeoDataFrame:
    df = pd.read_excel(path)

    columns = [
        "Latitude",
        "Longitude",
        "Injury.severity",
        "Victim.gender",
        "Incident.month",
        "Incident.year",
        "Shark.common.name",
        "Victim.activity",
        "Provoked/unprovoked",
    ]

    df = df[columns].copy()

    df = df.rename(
        columns={
            "Latitude": "lat",
            "Longitude": "long",
            "Victim.gender": "sex",
            "Shark.common.name": "species",
            "Victim.activity": "activity",
            "Provoked/unprovoked": "provoked",
        }
    )

    df["date"] = pd.to_datetime(
        df[["Incident.year", "Incident.month"]]
        .rename(columns={"Incident.year": "year",
                         "Incident.month": "month"})
        .assign(day=1),
        errors="coerce",
    )

    df["is_fatal"] = df["Injury.severity"].apply(
        lambda x: 1 if str(x).lower() == "fatal" else 0
    )

    df = df.dropna(subset=["lat", "long"])

    df = df[df["Incident.year"] >= 2015]

    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.long, df.lat),
        crs="EPSG:4326"
    )

    gdf = gpd.GeoDataFrame(
        gdf[
            [
                "geometry",
                "lat",
                "long",
                "is_fatal",
                "sex",
                "date",
                "species",
                "activity",
                "provoked",
            ]
        ].copy()
    )

    return gdf

In [ ]:
def read_croc_data(path: str) -> gpd.GeoDataFrame:
    df = pd.read_csv(path)

    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.long, df.lat),
        crs="EPSG:4326"
    )

    gdf["species"] = "crocodile"
    gdf["activity"] = None
    gdf["provoked"] = None

    gdf = gpd.GeoDataFrame(
        gdf[
            [
                "geometry",
                "lat",
                "long",
                "is_fatal",
                "sex",
                "date",
                "species",
                "activity",
                "provoked",
            ]
        ].copy()
    )

    return gdf

In [ ]:
def read_population_data(path: str) -> gpd.GeoDataFrame:
    df = pd.read_csv(path)
    pop_density_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(
        df.x, df.y))
    pop_density_gdf = pop_density_gdf.to_crs("EPSG:4326")
    return pop_density_gdf

In [ ]:
data_croc = read_croc_data("../data/croc_attacks.csv")
data_shark = read_shark_data("../data/shark_attacks.xlsx")
pop_density_gdf = gpd.read_file("../data/population_density.gpkg")

all_attacks = pd.concat([data_croc, data_shark], ignore_index=True)
data_combined = gpd.GeoDataFrame(all_attacks, geometry="geometry", crs="EPSG:4326")

Opis statystyk poniżej

In [ ]:
stats = data_combined.groupby('species')['is_fatal'].agg(['count', 'sum', 'mean'])
stats.columns = ['Liczba ataków', 'W tym śmiertelne', 'Śmiertelność (%)']
stats['Śmiertelność (%)'] = (stats['Śmiertelność (%)'] * 100).round(2)
display(stats)

Co robi ta funkcja co tworzy mape pełen opis

In [ ]:

output_file = "mapa_australia.html"
m = folium.Map(
    location=[-25.2744, 133.7751],
    zoom_start=4,
    tiles="cartodbpositron",
)

fg_croc_fatal = folium.FeatureGroup(name="Ataki krokodyli: Śmiertelne")
fg_croc_non_fatal = folium.FeatureGroup(name="Ataki krokodyli: Nieśmiertelne")
fg_shark_fatal = folium.FeatureGroup(name="Ataki rekinów: Śmiertelne")
fg_shark_non_fatal = folium.FeatureGroup(name="Ataki rekinów: Nieśmiertelne")
fg_population = folium.FeatureGroup(name="Gęstość zaludnienia")

croc_icon_path = "../icons/crocodile.png"
shark_icon_path = "../icons/shark.png"

for _, row in data_combined.iterrows():
    popup_info = ""
    for col in data_combined.columns:
        if col != "geometry":
            value = row[col]
            if pd.notna(value) and value != "" and value is not None:
                popup_info += f"<b>{col}:</b> {value}<br>"

    is_shark = "shark" in str(row.get("species", "")).lower()
    icon_path = shark_icon_path if is_shark else croc_icon_path

    if row["is_fatal"] == 1:
        bg_color = "#d32f2f"
        target_group = fg_shark_fatal if is_shark else fg_croc_fatal
    else:
        bg_color = "#f57c00"
        target_group = fg_shark_non_fatal if is_shark else fg_croc_non_fatal

    icon_html = f"""
        <div style="background-color: {bg_color}; border-radius: 50%; width: 24px; height: 24px;
                    display: flex; justify-content: center; align-items: center; border: 2px solid white;
                    box-shadow: 0 0 5px rgba(0,0,0,0.5);">
            <img src="{icon_path}" style="width: 16px; height: 16px;">
        </div>"""

    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_info, max_width=300),
        icon=folium.DivIcon(html=icon_html, icon_size=(24, 24), icon_anchor=(12, 12))
    ).add_to(target_group)

if pop_density_gdf is not None and not pop_density_gdf.empty:
    pop = pop_density_gdf.copy().to_crs("EPSG:4326")

    possible_names = ["population_density", "pop_density", "density", "value"]
    field = next((n for n in possible_names if n in pop.columns), pop.select_dtypes(include=['number']).columns[0])

    vals = pop[field].dropna()
    cmap = LinearColormap(
        ["#ffffb2", "#fecc5c", "#fd8d3c", "#f03b20", "#bd0026"],
        vmin=float(vals.min()), vmax=float(vals.max()),
        caption="Gęstość zaludnienia"
    )

    geom_types = set(pop.geometry.geom_type)
    if any("polygon" in gt.lower() for gt in geom_types):
        pop['geometry'] = pop['geometry'].simplify(0.01)

        folium.GeoJson(
            pop.to_json(),
            style_function=lambda feature: {
                "fillColor": cmap(feature["properties"].get(field)) if feature["properties"].get(field) else "#ffffff00",
                "color": "black", "weight": 0.3, "fillOpacity": 0.6
            },
            name="Gęstość zaludnienia (obszary)"
        ).add_to(fg_population)
    else:
        for _, prow in pop.iterrows():
            centroid = prow.geometry.centroid
            folium.CircleMarker(
                location=[centroid.y, centroid.x],
                radius=5, fill=True, fill_color=cmap(prow[field]),
                color=None, fill_opacity=0.7,
                popup=f"{field}: {prow[field]}"
            ).add_to(fg_population)

    cmap.add_to(m)

fg_croc_fatal.add_to(m)
fg_croc_non_fatal.add_to(m)
fg_shark_fatal.add_to(m)
fg_shark_non_fatal.add_to(m)
fg_population.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save(output_file)
m

opis mapy poniżej i funkcji

In [ ]:
from folium.plugins import HeatMap

mapa_ciepla = folium.Map(
    location=[-25.2744, 133.7751],
    zoom_start=4,
    tiles="cartodbpositron"
)

points = [[row.geometry.y, row.geometry.x] for index, row in data_combined.iterrows()]

HeatMap(
    data=points,
    radius=25,
    blur=20,
    min_opacity=0.4,
    gradient={
        0.2: 'blue',
        0.4: 'cyan',
        0.6: 'lime',
        0.8: 'yellow',
        1.0: 'red'
    },
    name="Mapa Ciepła Ataków"
).add_to(mapa_ciepla)

if 'pop_density_gdf' in locals() and pop_density_gdf is not None:
    fg_pop = folium.FeatureGroup(name="Gęstość zaludnienia (kontury)")
    pop = pop_density_gdf.copy().to_crs("EPSG:4326")

    if any("polygon" in gt.lower() for gt in set(pop.geometry.geom_type)):
         pop['geometry'] = pop['geometry'].simplify(0.05)

    folium.GeoJson(
        pop.to_json(),
        style_function=lambda x: {
            'fillColor': '#4B0082',
            'color': '#2E0854',
            'weight': 0.5,
            'fillOpacity': 0.1
        },
        name="Kontury obszarów gęstości"
    ).add_to(fg_pop)
    fg_pop.add_to(mapa_ciepla)

folium.LayerControl().add_to(mapa_ciepla)

mapa_ciepla